### first, why X@A can be splitted into [XA1, XA2] where we partition the A in the column parallel fashion. [1]

[1] Megatron-LM: Training Multi-Billion Parameter Language Models Using Model Parallelism

In [97]:
import torch
x = torch.randn(2, 5)
a = torch.randn(5, 20)

In [98]:
xa1 = x@a[:,:10]
xa2 = x@a[:,10:]

In [99]:
(x@a).shape, xa1.shape, xa2.shape

(torch.Size([2, 20]), torch.Size([2, 10]), torch.Size([2, 10]))

In [15]:
# thus, in x@a, a can be partitioned in column. It is similar to the FFN neurons' parallels claim in DynaBERT
x@a == torch.cat((xa1, xa2), dim=-1)

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True]])

In [95]:
# in x@a, a is linear transformation, thus a can be partitioned and x can not.
torch.concat((x[:1, :] @ a, x[1:, :] @ a), dim = 0) == x@a

tensor([[False, False, False, False, False,  True,  True,  True,  True, False,
          True, False,  True, False,  True, False, False,  True, False, False],
        [ True,  True,  True, False,  True, False, False,  True, False, False,
          True, False,  True, False, False,  True, False, False,  True,  True]])

In [165]:
# todo. 费解的地方，为什么x@a中，拆开a没有精度问题，拆开x会有
# torch.backends.cudnn.deterministic = True
print(float(x.matmul(a)[0][0]))
print(float((x@a)[0][0]))
print(float(torch.concat((x[:1,:]@a,x[1:,:]@a),dim=0)[0][0]))
print(float(torch.concat((x@a[:,:1],x@a[:,1:]),dim=-1)[0][0]))

0.3176284730434418
0.3176284730434418
0.3176284432411194
0.3176284730434418


### how about adding gelu

In [18]:
# def gelu(x):
#     '''
#     Two way to implements GELU, torch.nn use the second one:
#     0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))
#     or
#     0.5 * x * (1. + torch.erf(torch.sqrt(x, 2))) 
#     '''
#     return .5 * x * (1. + torch.erf(x / math.sqrt(2.)))
gelu = torch.nn.GELU()

In [83]:
# gelu(xa1 + xa2) != gelu(xa1) + gelu(xa2)
# because gelu is non-linear, its output depends on all x
gelu(xa1) + gelu(xa2) != gelu(xa1 + xa2)

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [40]:
# gelu()
torch.cat((gelu(xa1), gelu(xa2)), dim=-1) == gelu(x@a)

tensor([[True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True, True, True,
         True, True, True, True, True, True, True, True]])

# Conclutions
- x@a 的本质，想象x是seq，a是线性变换，x有很多行，a有很多列，其实就是对seq的每一行施加线性变换，让seq的每一行变长；
- 在这个过程中，a是可以按列拆分的，之后再AllReduce还原回来，但是x由于需要过non-linear，所以不能按行拆分；

# Future
- consider bias